# STICS-Inspired Maize Crop Model Implementation

## 1. Thermal Time (TT)
We accumulate daily thermal time above a base temperature $T_{\rm base}$ to drive phenology:

$$
\Delta TT_t = \max\bigl(0,\;\tfrac{T_{\min,t} + T_{\max,t}}{2} - T_{\rm base}\bigr),
\quad
TT_t = \sum_{u=1}^t \Delta TT_u
$$

- $T_{\min,t},\;T_{\max,t}$: daily min/max air temperature (°C)  
- $T_{\rm base} = 8.0$ °C  

## 2. Soil Water Balance
For each soil layer $i$ with thickness $z_i$ (m), field capacity $FC_i$, wilting point $WP_i$:

1. **Depth of water** (mm):  
   $$
     W_{i,t} = \theta_{i,t}\,z_i \times 1000,\quad \theta_{i,0} = FC_i
   $$

2. **Potential ET** via Hargreaves:  
   $$
     PET_t = 0.0023\,\sqrt{T_{\max,t}-T_{\min,t}}\,
             \bigl(\tfrac{T_{\max,t}+T_{\min,t}}{2}+17.8\bigr)\,R_{\rm rad,t}
   $$

3. **Actual ET** limited by available water fraction:  
   $$
     f_{i,t} = \mathrm{clip}\!\Bigl(\frac{W_{i,t}/1000 - WP_i}{FC_i - WP_i},0,1\Bigr),\quad
     ET_{i,t} = PET_t \,f_{i,t},\quad
     ETa_t = \sum_i ET_{i,t}
   $$

4. **Update water**:  
   $$
     W_{i,t+1} = \max(0,\,W_{i,t}-ET_{i,t}) + P_t\frac{z_i}{\sum_j z_j}
   $$

## 3. Leaf Area Index & Biomass Growth
Define a fixed daily LAI increase until half the cycle:

$$
\Delta \mathrm{LAI}_{\max} = \frac{\mathrm{Max\_LAI}}{\mathrm{Cycle\_days}/2},\quad
\Delta \mathrm{LAI}_t = 
\begin{cases}
\min(\Delta \mathrm{LAI}_{\max},\,\mathrm{Max\_LAI}), & t=0,\$$4pt]
\min(\mathrm{LAI}_{t-1} + \Delta \mathrm{LAI}_{\max},\,\mathrm{Max\_LAI}), & (t - t_{\rm sow})\le \tfrac{\mathrm{Cycle\_days}}{2},\$$4pt]
\mathrm{LAI}_{t-1}, & \text{otherwise}.
\end{cases}
$$

Intercepted PAR:
$$
IPAR_t = R_{\rm rad,t}\,\bigl(1 - e^{-k\,\mathrm{LAI}_t}\bigr),\quad k=0.65
$$

Daily biomass increment (g m⁻² d⁻¹):
$$
dB_t = \underbrace{\mathrm{LUE}}_{\rm g\,MJ^{-1}}
       \times IPAR_t \times D \times \mathrm{NNI}_t
$$
- **LUE** = Light Use Efficiency  
- **D** = planting density (plants m⁻²)  
- **NNI$_t$** = Nitrogen Nutrition Index  

Then:
$$
B_{t+1} = B_t + dB_t,\quad
\mathrm{LAI}_{t+1} = \mathrm{LAI}_t + \Delta\mathrm{LAI}_t \times \mathrm{NNI}_t
$$

## 4. Nitrogen Uptake & Nutrition Index
1. **Daily N demand** (kg ha⁻¹):  
   $$
     N_{\rm demand,t} = dB_t\,(\tfrac{\rm g}{m^2})
                        \times 10 \times 0.03
   $$
2. **Supply** from soil:  
   $$
     \mathrm{Supply}_t = \sum_i N_{\min,i,t},\quad
     U_t = \min(\mathrm{Supply}_t,\,N_{\rm demand,t})
   $$
   $$
     N_{\min,i,t+1} = N_{\min,i,t} - U_t \frac{z_i}{\sum_j z_j}
   $$
3. **Mineralization**:  
   $$
     N_{\min,i,t+1} \;+\!=\; \frac{\text{mineralization\_rate}}{n_{\rm layers}}
   $$
4. **Leaching** (10% of percolated N):  
   $$
     \mathrm{Percol}_i = \max(0,\,W_{i,t}/1000 - FC_i),\;
     \mathrm{Leach}_i = 0.1 \times \mathrm{Percol}_i \frac{N_{\min,i,t}}{W_{i,t}+\epsilon}
   $$
5. **Nitrogen Nutrition Index (NNI)**:  
   $$
     \mathrm{NNI}_t = \min\Bigl(1,\;\frac{\mathrm{Supply}_t}{N_{\rm demand,t}+\epsilon}\Bigr)
   $$

## 5. Outputs & Summary
- **Daily**: $TT_t,\;\mathrm{LAI}_t,\;B_t,\;SWC_t,\;N_{\min,t},\;ETa_t$.  
- **Summary**:
  - Final $B_{\rm final}$ → Grain ≈ 50%.  
  - Total ET: $\sum_t ETa_t$.  
  - Total N uptake: initial N − final $N_{\min}$.



# Maize-Specific Parameters in Our STICS-Inspired Model

1. **Phenology & Development**  
   - `FAO_Maturity_Group` (from variety CSV)  
   - `Cycle_days` (total days from sowing to maturity)  
   - Vegetative phase = ½ `Cycle_days` (used for LAI build-up)  
   - Base temperature: $T_{\text{base}} = 8^\circ\mathrm{C}$

2. **Radiation‐Use & Canopy**  
   - `Light_Use_Efficiency` (LUE, g MJ⁻¹)  
   - `Max_LAI` (maximum leaf area index)  
   - Extinction coefficient $k = 0.65$

3. **Planting Configuration**  
   - `density` (plants m⁻²)

4. **Nitrogen Dynamics**  
   - `N_Uptake_Efficiency` (from variety CSV)  
   - Tissue N concentration = 3% of daily biomass increment  
   - Mineralization rate = 0.5 kg N/ha/day  
   - Leaching factor = 0.1 (10% of percolated N lost)  
   - Grain partitioning = 50% of final biomass → grain yield

5. **Root & Soil Interaction**  
   - `Root_Depth` (m) → uniform root_fraction across layers  
   - Soil layers defined by `soil_layers` (depth_top, depth_bottom, FC, WP)

All these parameters together define maize growth, phenology, light capture, and N interactions in our simplified STICS model.

# Simplifications Compared to Full STICS

1. **Single‐Crop Focus**  
   - This implementation only handles maize; STICS can simulate many crops with crop-specific modules.

2. **Fixed Root Distribution**  
   - We assume a uniform root fraction across soil layers.  
   - Full STICS uses dynamic root-profile growth and water/nutrient extraction curves.

3. **Constant RUE & Extinction**  
   - Light Use Efficiency (LUE) and extinction coefficient $k$ are fixed.  
   - STICS adjusts RUE by developmental stage, water status, and nitrogen nutrition.

4. **Simplified Phenology**  
   - Phenological progression driven solely by thermal time above a base temperature.  
   - STICS includes photoperiod sensitivity and vernalization effects for some crops.

5. **One‐Parameter Mineralization & Leaching**  
   - We use a constant mineralization rate and a fixed 10% leaching factor.  
   - STICS models organic matter turnover, nitrification, denitrification, and dynamic leaching hydraulics.

6. **Bulk Soil Water Balance**  
   - Soil water is tracked by layer with a Hargreaves PET and fraction‐based ETa.  
   - STICS solves Richards’ equation for water flow and simulates dynamic soil hydraulic properties.

7. **Simple N Uptake**  
   - Uptake = min(demand, supply) split by root fraction.  
   - STICS uses Michaelis–Menten kinetics, root growth dynamics, and diffusion mass-flow partitioning.

8. **No Carbon Allocation Complexity**  
   - Biomass partitioned 50:50 to grain at maturity.  
   - STICS has dynamic allocation to leaves, stems, roots, grains based on phenology and stress.

9. **No Pest/Disease or Management Modules**  
   - Excludes damage from biotic stresses and detailed irrigation/fertilization timing rules.  
   - Full STICS includes detailed management scenarios, intercropping, and stress interactions.

10. **Coarse Time Step & No Tilting**  
    - Daily timestep, no sub‐daily processes.  
    - STICS can run hourly and simulate canopy energy balance, detailed soil thermal regimes, and sowing/emergence mechanics.

---

*These simplifications make our Python version much lighter and faster to run, while capturing the core crop–water–nitrogen feedbacks for maize.*  
